In [1]:
from scipy import spatial
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import random
import time
import math
tokenizer = RegexpTokenizer(r'\w+')


## Now with our data

In [2]:
ItemsDF = pd.read_excel('Items_IS.xlsx') #header=None, skiprows=1)
#df.head(10)
#dfd = df.head(10)
dfd = ItemsDF[['ItemId','VariableId','Text']]

In [31]:
(17838*(17838-1))/2

159088203.0

## Create a new column in your existing dataframe

## Import the GloVE

In [3]:
import pandas as pd
import csv

words = pd.read_table('glove.840B.300d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

## Make the vec function

In [4]:
def vec(w):
    return words.loc[w].as_matrix()

## Create an overlapping dictionary

In [5]:
import re
all_words = [word for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
unique = (set(all_words))

In [6]:
all_words1 = [word.lower() for sentence in ItemsDF['Text'] for word in re.split('\W', sentence)]
lower = (set(all_words))

In [7]:
glove_words = list(words.index.values)

In [8]:
a = unique
b = glove_words
c = []
for e in b:
    if e in a:
        c.append(e)
    

In [9]:
dict1 = {}
for word in c:
    vword = vec(word)
    vector = {word : vword }
    dict1.update(vector)


In [10]:
Embeddings1 = dict1

In [11]:
stop_words = set(stopwords.words('english'))

## Weighted Words

In [12]:
idf= pd.read_csv('TDNews.csv')
idf.head(5)

,TermID,TermName,StemID,StemName,DocumentID,Frequency
0,74169,committee,64277,committe,846,1
1,72262,co,62921,co,846,1
2,79071,craig,67593,craig,846,3
3,95581,easy,79854,easi,91,1
4,93002,drastic,77822,drastic,846,1


## n and N

In [13]:
idff = idf.groupby(['TermName','TermID'], as_index = False, sort = False)['DocumentID'].count()
df = idf.groupby(['DocumentID'],as_index = False, sort = False)['Frequency'].count().count()
df


DocumentID    161754
Frequency     161754
dtype: int64

### Log2(N/n)

In [14]:
var = df['DocumentID']
var
idff['weight'] = idff['DocumentID'].apply(lambda x: math.log((var/x),2))


In [15]:
idff.head(10)

,TermName,TermID,DocumentID,weight
0,committee,74169,10938,3.886381
1,co,72262,37710,2.100782
2,craig,79071,2886,5.808586
3,easy,95581,9699,4.059822
4,drastic,93002,705,7.841962
5,continue,76491,22324,2.857134
6,employee,98642,9532,4.084879
7,experimental,103047,1101,7.198843
8,hard,125402,22819,2.825494
9,denying,86587,1090,7.213329


## Now make the dictionary
#### Update the dictionary with unfound words as the keys and AVG as the values

In [16]:
print((idff['weight']).mean())
print((idff['weight']).median())

15.372384792109694
16.30344186337058


In [17]:
weightdict =  dict(zip(idff.TermName,idff.iloc[:,3]))
a1 = unique
b1 = weightdict
c1 = []
for e in a1:
    if e not in b1:
        c1.append(e)

In [18]:
tempmean = 0
for word in all_words1:
    if word in weightdict.keys():
        tempmean = tempmean + weightdict[word]
mean = tempmean/len(all_words1)

In [19]:
for word in c1:
    weightdict.update({word:mean})

In [20]:
weightdict

{'committee': 3.8863805166102887,
 'co': 2.1007823333415523,
 'craig': 5.80858627887938,
 'easy': 4.059821570523337,
 'drastic': 7.841962416084423,
 'continue': 2.8571339323202327,
 'employee': 4.0848786271803075,
 'experimental': 7.1988431098062104,
 'hard': 2.8254939142213593,
 'denying': 7.21332944370629,
 'credit': 1.15127587715498,
 'commerce': 4.481667881400011,
 'expert': 5.2841986467669715,
 'could': 1.098641495535761,
 'facts': 5.730268078684093,
 'greatest': 5.786264610681037,
 'has': 0.36621748866976084,
 'husband': 5.541890630926099,
 'it': 0.14691433541401436,
 'changes': 3.4843605007212894,
 'fairly': 5.438481948227978,
 'didnt': 2.603706758206076,
 'do': 1.4781849118565544,
 'companies': 1.4998909012103703,
 'industry': 1.9620620347823041,
 'harvard': 5.774499448305684,
 'conscience': 8.945889858752494,
 'even': 1.5703735647254293,
 'consumers': 3.077180210055075,
 'her': 2.9225073732821047,
 'insurers': 6.36533253715134,
 'lower': 3.1424674884426427,
 'copyright': 0.032

## Functions!

In [21]:
def calculateCosine1(W1,W2):
    #resultww = []
    result = 1 - spatial.distance.cosine(Embeddings1[W1], Embeddings1[W2])
    #if W1.lower() in weightdict:
        
    result = round(result, 2)
    return(result)

In [22]:
def calculateSentenceSim1(S1,S2):
    pairs = []
    for index1,word1 in enumerate(S1):
        for index2,word2 in enumerate(S2):
            pairs.append([index1+1, word1,index2+1,word2,calculateCosine1(word1,word2)*weightdict[word1]*weightdict[word2]])
    pairs = sorted(pairs, key=lambda x:x[4], reverse=True)
    
    # Now create the similarity vector
    tempPairsVector = []
    for index, row in enumerate(pairs):


        foundOne=False
        for tempRow in tempPairsVector:

            if tempRow[0] == row[0] or tempRow[2] == row[2]:
                foundOne=True
        if foundOne==False:
            tempPairsVector.append(row)

#Now create the vector of the up to 20 highest cosines
    sentenceSimVector = [] #[None] * 20
    for index,row in enumerate(tempPairsVector):
        sentenceSimVector.append(row[4])
    
#Pad the similarity vector with nulls    
    length = 20 - len(sentenceSimVector)
    for x in range(0, length):
        sentenceSimVector.append(None)            
    return(sentenceSimVector)

## Now with all the data

In [152]:
df_test = df.head(5)


DocumentID    161754
Frequency     161754
dtype: int64

## Set up the data

## Make a dictionary from the DataFrame

In [58]:
df_test = ItemsDF.head(20)

In [25]:
import re
new = {}
dictionary = {x[0]: { x[1] : x[2:] } for x in ItemsDF.itertuples(index=False)}
for key, item in dictionary.items():
    #print(key, item)
    for key2, item2 in item.items():
        itemslist = (list(item2))  
        words = ([word for sentence in itemslist for word in tokenizer.tokenize(sentence)])
        for word in words:
            if word not in Embeddings1.keys():
                words.remove(word)
        #lowering is something we should think about
        filtered_sentence = [w for w in words if not w in stop_words]
        #filtered_sentence = [w.lower() for w in words if not w in stop_words]
        #print(key, key2)
        new[key] =  [key2, filtered_sentence]
new

{1: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'understanding',
   'module',
   'integrates',
   'modules',
   'systems']],
 2: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'ability',
   'ask',
   'penetrating',
   'questions',
   'module']],
 3: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'improved',
   'knowledge',
   'module']],
 4: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'knowledge',
   'module',
   'training',
   'documents',
   'end',
   'users']],
 5: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant'

In [66]:
import re
newtest = {}
dictionary = {x[0]: { x[1] : x[2:] } for x in df_test.itertuples(index=False)}
for key, item in dictionary.items():
    #print(key, item)
    for key2, item2 in item.items():
        itemslist = (list(item2))  
        words = ([word for sentence in itemslist for word in tokenizer.tokenize(sentence)])
        for word in words:
            if word not in Embeddings1.keys():
                words.remove(word)
        filtered_sentence = [w for w in words if not w in stop_words]
        #print(key, key2)
        newtest[key] =  [key2, filtered_sentence]

{1: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'understanding',
   'module',
   'integrates',
   'modules',
   'systems']],
 2: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'ability',
   'ask',
   'penetrating',
   'questions',
   'module']],
 3: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'improved',
   'knowledge',
   'module']],
 4: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant',
   'William',
   'increased',
   'knowledge',
   'module',
   'training',
   'documents',
   'end',
   'users']],
 5: [14,
  ['During',
   'Purchasing',
   'module',
   'implementation',
   'project',
   'interactions',
   'Consultant'

In [32]:
table = []
start = time.time()
random.seed(1)
for key, value in new.items():
    for key2, values2 in new.items():
        if key >= key2:
            continue
        #if key2 >= key:
        if value[0]!=values2[0]:
            foundnum=random.randint(1,3167)
            if foundnum == 1:
                table.append([key, key2,value[0]* values2[0],0, calculateSentenceSim1(value[1],values2[1])])
        else:
            table.append([key, key2,value[0]* values2[0],1, calculateSentenceSim1(value[1],values2[1])])


In [67]:
#not to be used
tabletest = []
start = time.time()
random.seed(1)
for key, value in newtest.items():
    for key2, values2 in newtest.items():
        if key >= key2:
            continue
        if value[0]!=values2[0]:
            foundnum=random.randint(1,100000)
            if foundnum == 1:
                table.append([key, key2,value[0]* values2[0],0, calculateSentenceSim1(value[1],values2[1])])
        else:
            table.append([key, key2,value[0]* values2[0],1, calculateSentenceSim1(value[1],values2[1])])
       
    if len(table) > 10000:
        print(time.time() - start)

0.21953582763671875
0.38275575637817383
0.5477049350738525
0.6963858604431152
0.8742086887359619
1.0662457942962646
1.2448749542236328
1.3829541206359863
1.5241920948028564
1.642535924911499
1.6997849941253662
1.768155813217163
1.8907039165496826
1.986595869064331
2.132564067840576
2.211120843887329
2.224491834640503
2.2458279132843018
2.2599220275878906
2.260037899017334


[]

### 50212 trues

In [33]:
list1 = []
list1.append([])
list1 = []
list1.append([])
z = 0
for x in table:
    list2 = []
    for y in x[4]:
        
        #print(y)
        #print(x[0],x[1],x[2], x[3],y)
        list2.append(y)
        #for m in list2:
            #list3.append(m)
            #list3[z].extend(m)
    list1[z].append(x[0])
    list1[z].append(x[1])
    list1[z].append(x[2])
    list1[z].append(x[3])
    list1[z].extend(list2)
    list1[z]
    z += 1
    list1.append([])
list1[0]
#list3

[1,
 2,
 196,
 1,
 90.67004846841427,
 83.51282078919309,
 83.51282078919309,
 66.90128468486736,
 18.08809080725074,
 15.173966206060578,
 12.528993366088304,
 12.489277724104978,
 7.179484606412504,
 5.870747074852751,
 5.870747074852751,
 5.870747074852751,
 5.870747074852751,
 2.0489165883909246,
 None,
 None,
 None,
 None,
 None,
 None]

In [34]:
tableDF2 = pd.DataFrame(list1)


In [35]:
tableDF2 = tableDF2.rename(columns = {0:'ItemId1', 1:'ItemId2',2:'VId',3:'Target'})
tableDF2.head(7)

,ItemId1,ItemId2,VId,Target,4,5,6,7,8,9,...,57,58,59,60,61,62,63,64,65,66
0,1.0,2.0,196.0,1.0,90.670048,83.512821,83.512821,66.901285,18.088091,15.173966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,3.0,196.0,1.0,90.670048,83.512821,83.512821,66.901285,26.188625,15.173966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,4.0,196.0,1.0,90.670048,83.512821,83.512821,66.901285,26.188625,15.173966,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,5.0,196.0,1.0,90.670048,83.512821,66.901285,49.681743,28.967924,26.188625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,6.0,196.0,1.0,90.670048,83.512821,83.512821,66.901285,39.304760,15.652012,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1.0,28108.0,387576.0,0.0,39.029185,34.419165,26.188625,16.886617,7.956144,4.871312,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1.0,42807.0,333060.0,0.0,17.439168,15.522113,14.568079,10.441964,8.255506,7.343539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Downsample

In [36]:
Final = tableDF2.iloc[:,0:24]

In [38]:
Final.to_csv('UseMultipliedWordWeightGloveVec.csv')